### 생성된 CSV를 벡터에 저장한다.

In [4]:
# Q&A CSV파일을 통해서 벡터 임베딩 

import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# CSV 읽기
df = pd.read_csv('./data/qa_dataset.csv')
documents = [
    f"질문: {row['user_input']}\n답변: {row['reference']}"
    for _, row in df.iterrows()
]
questions = df['user_input'].tolist()
answers = df['reference'].tolist()

# 임베딩 및 벡터DB 저장
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_texts(
    texts=documents,
    embedding=embedding,
    metadatas=[{"question": q, "answer": a} for q, a in zip(questions, answers)],
)

print("Chroma 벡터DB 저장 완료!");

Chroma 벡터DB 저장 완료!


### 벡터 저장소를 활용한 챗봇을 완성한다.

In [5]:
# 생성된 벡터스토어를 활용하는 Gradio 기반 챗봇 생성

import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "다음 컨텍스트를 참고하여 사용자의 질문에 답변하세요."),
    ("human", "컨텍스트: {context}\n질문: {question}")
])

# LLM 모델 정의
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)
chain = prompt | model | StrOutputParser()

def rag_chatbot(message, history):
    results = vectorstore.similarity_search(message, k=3)
    if results:
        context = "\n".join([doc.page_content for doc in results])
        # LLM에 context와 question 전달
        response = chain.invoke({
            "context": context,
            "question": message
        })
        return response
    else:
        return "적절한 답변을 찾지 못했습니다."

chatbot = gr.ChatInterface(
    fn=rag_chatbot,
    title="QA RAG 챗봇",
    description="QA 데이터셋 기반 RAG 챗봇입니다.",
    analytics_enabled=False,
    type="messages"
)

chatbot.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [3]:
# 챗봇 중지
chatbot.close()

Closing server running on port: 7862
